In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from typing import Callable, Union
from mytorch import dataiters
from mytorch.utils.goodies import default_eval, Timer

In [2]:
with Path('../resources/6.1.X.np').open('rb') as f:
    X = np.load(f)
    
with Path('../resources/6.1.Y.np').open('rb') as f:
    Y = np.load(f)

Y = Y.reshape(-1, 1)
X.shape, Y.shape, X.dtype, Y.dtype

((25000, 10002), (25000, 1), dtype('float64'), dtype('int64'))

In [3]:
X.dtype, Y.dtype

(dtype('float64'), dtype('int64'))

In [4]:
n_words = X.shape[1]
n_docs = X.shape[0]

In [5]:
# Shuffle the dataset
p = np.random.permutation(len(X))
X = X[p]
Y = Y[p]

In [6]:
# Split the dataset 
X_train, X_valid = X[:(int(0.8*n_docs))], X[(int(0.8*n_docs)):]
Y_train, Y_valid = Y[:(int(0.8*n_docs))], Y[(int(0.8*n_docs)):]

X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

((20000, 10002), (5000, 10002), (20000, 1), (5000, 1))

In [7]:
class NonLin(torch.nn.Module):

    def __init__(self, n_words):
        super().__init__()
        
        self.t1 = torch.nn.Linear(n_words, n_words//20) # 10 + 10
        self.t3 = torch.nn.Linear( n_words//20, 1) # 10 + 1
        
    def forward(self, x):
        x = x.float()
        x = F.sigmoid(self.t1(x))
#         x = F.sigmoid(self.t2(x))
        x = torch.softmax(self.t3(x), dim=1)
        return x


In [8]:
m = NonLin(n_words)
lfn = torch.nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(m.parameters(), lr=100)


In [9]:
epochs = 100

In [10]:
from mytorch.loops import simplest_loop

In [11]:
def simplest_loop(epochs: int,
                  data: dict,
                  opt: torch.optim,
                  loss_fn: torch.nn,
                  train_fn: Callable,
                  predict_fn: Callable,
                  device: Union[str, torch.device] = torch.device('cpu'),
                  data_fn: classmethod = dataiters.SimplestSampler,
                  eval_fn: Callable = default_eval) -> (list, list, list):
    """
        A fn which can be used to train a language model.

        The model doesn't need to be an nn.Module,
            but have an eval (optional), a train and a predict function.

        Data should be a dict like so:
            {"train":{"x":np.arr, "y":np.arr}, "val":{"x":np.arr, "y":np.arr} }

        Train_fn must return both loss and y_pred

        :param epochs: number of epochs to train for
        :param data: a dict having keys train_x, test_x, train_y, test_y
        :param device: torch device to create new tensor from data
        :param opt: optimizer
        :param loss_fn: loss function
        :param train_fn: function to call with x and y
        :param predict_fn: function to call with x (test)
        :param data_fn: a class to which we can pass X and Y, and get an iterator.
        :param eval_fn: (optional) function which when given pred and true, returns acc
        :return: traces
    """

    train_loss = []
    train_acc = []
    valid_acc = []
    lrs = []

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = []
        per_epoch_tr_acc = []

        # Train
        with Timer() as timer:

            # Make data
            trn_dl, val_dl = data_fn(data['train']), data_fn(data['valid'])

            for x, y in tqdm(trn_dl):
                opt.zero_grad()

                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.float, device=device)

                y_pred = train_fn(_x)
                loss = loss_fn(y_pred, _y)

                per_epoch_tr_acc.append(eval_fn(y_pred=y_pred, y_true=_y).item())
                per_epoch_loss.append(loss.item())

                loss.backward()
                opt.step()

        # Val
        with torch.no_grad():

            per_epoch_vl_acc = []
            for x, y in tqdm(val_dl):
                _x = torch.tensor(x, dtype=torch.long, device=device)
                _y = torch.tensor(y, dtype=torch.long, device=device)

                y_pred = predict_fn(_x)

                per_epoch_vl_acc.append(eval_fn(y_pred, _y).item())

        # Bookkeep
        train_acc.append(np.mean(per_epoch_tr_acc))
        train_loss.append(np.mean(per_epoch_loss))
        valid_acc.append(np.mean(per_epoch_vl_acc))

        print("Epoch: %(epo)03d | Loss: %(loss).5f | Tr_c: %(tracc)0.5f | Vl_c: %(vlacc)0.5f | Time: %(time).3f min"
              % {'epo': e,
                 'loss': float(np.mean(per_epoch_loss)),
                 'tracc': float(np.mean(per_epoch_tr_acc)),
                 'vlacc': float(np.mean(per_epoch_vl_acc)),
                 'time': timer.interval / 60.0})

    return train_acc, valid_acc, train_loss


In [ ]:
traces = simplest_loop(
    epochs = epochs,
    data = {"train": {"x": X_train, "y": Y_train}, "valid": {"x": X_valid, "y": Y_valid}},
    loss_fn = lfn,
    train_fn = m,
    predict_fn = m,
    opt = opt
)

  0%|          | 0/311 [00:00<?, ?it/s]

/home/priyansh/Dev/perm/conda/envs/main/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
Y_valid.mean()